# IGC Benchmark

In order to assess the relevancy of IGC attributions compared to other naive
dataset-wise attribution methods, we propose synthetic experiments. We first
define some localized image statistics computed on random images, and then try
to recover the generating masks/rules from the pairs of original images and
computed statistics. To make this procedure feasible, generated random images
are designed to respect the spatial frequency distribution of natural images
(i.e. having some spatial redundancy). For more details, look at the _Benchmark_
section of the original IGC [paper](http://arxiv.org/abs/2404.13910).



## IGC from original localized image statistic functions

### Load the dataset

In [ ]:
from igcsimulation.simulation_1v0 import Dataset

# Experiments A, B, C, D (mask_name, imst_name, imst_kwargs)
expe = ('comb_01', 'w_sum', None)
#expe = ('ccat_01', 'max_mean_bin', None)
#expe = ('ccat_02', 'max_sim_rand00_bin', {'probs': (0.0, 0.5)})
#expe = ('ccat_03', 'argmax_sim_rand01_bin', {'permute': True})

img_size = 64
n_samples = (1000, 1000)
# n_samples = (100000, 100000)  # Paper's values
fft_slope = -1.2  # for natural images

dataset = Dataset(expe[0], expe[1], img_size, n_samples, fft_slope, expe[2])

### Init the model

In [ ]:
from igcsimulation.simulation_1v0 import Model

device = 'cpu'
# device = 'cuda'

model = Model(dataset, model_name='sim_1v0_a100', device=device)

### Compute IGC attributions

In [ ]:
_ = model.int_grad_corr(x_0=8, y_idx=0, n_steps=512, batch_size=8)

### Compute attributions with other methods

In [ ]:
_ = model.int_grad_auto_corr(x_0=8, y_idx=0, n_steps=512, batch_size=8)
_ = model.int_grad_mean_std(x_0=8, y_idx=0, n_steps=512, batch_size=8)
_ = model.naive_corr(y_idx=0, batch_size=100)
_ = model.naive_ttest(y_idx=0, batch_size=100)
_ = model.bsl_shap_corr(
    x_0=8, y_idx=0, n_iter=2, x_0_batch_size=8, n_x=n_samples[0]//10)

### Compare attributions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# (label, filename)
attributions = (
    ('IGC', 'int_grad_corr.npz'),
    ('IGaC', 'int_grad_auto.npz'),
    ('IG mean', 'int_grad_mean.npz'),
    ('IG SD', 'int_grad_std.npz'),
    ('naive correlation', 'corr.npz'),
    ('naive ttest', 'ttest.npz'),
    ('BSC', 'bsl_shap_corr.npz'),
)

fig, axs = plt.subplots(
    1, 7, figsize=(14, 2),
    gridspec_kw = {'top': 0.85, 'bottom': 0.01, 'left': 0.01, 'right': 0.99})

for i, (label, filename) in enumerate(attributions):
    attr = np.load(model.get_result_path(filename))['data'][0]
    if label == 'naive ttest':
        attr = (attr < 0.001)*1.0 - 0.5  # p-value < 0.001

    ax = axs[i]
    ax.axis('off')
    ax.annotate(
        label, (0.5, 1.05), va='bottom', ha='center', xycoords='axes fraction')
    v_max = 1.25 * np.quantile(np.abs(attr), 0.99)
    ax.imshow(
        attr, cmap='RdBu_r', vmin=-1.0*v_max, vmax=v_max,
        extent=(-0.5, 0.5, -0.5, 0.5), interpolation='nearest')

## IGC from trained models

### Load the dataset

In [ ]:
from igcsimulation.simulation_1v0 import Dataset

# Experiments A, B, C, D (mask_name, imst_name, imst_kwargs)
expe = ('comb_01', 'w_sum', None)
#expe = ('ccat_01', 'max_mean_bin', None)
#expe = ('ccat_02', 'max_sim_rand00_bin', {'probs': (0.0, 0.5)})
#expe = ('ccat_03', 'argmax_sim_rand01_bin', {'permute': True})

img_size = 64
n_samples = (1000, 1000)
# n_samples = (100000, 100000)  # Paper's values
fft_slope = -1.2  # for natural images

dataset = Dataset(expe[0], expe[1], img_size, n_samples, fft_slope, expe[2])

### Init the model

In [ ]:
# For scalar statistics (experiments A, B, C)
from igcsimulation.model_1v0 import Model

# For categorical statistics (experiment D)
# from igcsimulation.model_cat_1v0 import Model

# ConvNeXt architecture
parameters = {
    'cvnx_sizes': (16, 32, 64, 128, 256),
    'cvnx_stem_kernel': 2,
    'lin_sizes': (128, 16),
}

# Simple multilayer perceptron 
# parameters = {
#     'cvnx_sizes': None,
#     'cvnx_stem_kernel': None,
#     'lin_sizes': (256, 128, 64, 32, 16),
# }

device = 'cpu'
# device = 'cuda'

model = Model(
    dataset, model_name='model_1v0_a100', trainable=True, device=device)

### Train the model

In [ ]:
model.train(n_epoch=1, batch_size=64)

### Compute IGC attributions

In [ ]:
_ = model.int_grad_corr(x_0=8, y_idx=0, n_steps=64, batch_size=8)

### Visualize IGC attributions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(
    1, 1, figsize=(2, 2),
    gridspec_kw = {'top': 0.85, 'bottom': 0.01, 'left': 0.01, 'right': 0.99})

attr = np.load(model.get_result_path('int_grad_corr.npz'))['data'][0]

ax.axis('off')
ax.annotate(
    'IGC (model)', (0.5, 1.05), va='bottom', ha='center',
    xycoords='axes fraction')
v_max = 1.25 * np.quantile(np.abs(attr), 0.99)
ax.imshow(
    attr, cmap='RdBu_r', vmin=-1.0*v_max, vmax=v_max,
    extent=(-0.5, 0.5, -0.5, 0.5), interpolation='nearest')